In [1]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Data Fetching

In [3]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_9012\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [4]:
data

,Spindle Speed,Feed (mm/min),Tool Dia. (mm),Length of cut actual(mm),Actual Width(mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
0,3000.0,10.0,1.0,11.0480,1.18700,49.3753,49.3694,0.0059,3.0,1.076451,0.0480,0.18700
1,3000.0,15.0,1.0,11.0640,1.00200,49.3694,49.3682,0.0012,4.5,0.327195,0.0640,0.00200
2,3000.0,20.0,1.0,11.0500,1.14500,49.3682,49.3659,0.0023,6.0,0.806629,0.0500,0.14500
3,3250.0,10.0,1.0,11.0565,1.14425,49.3659,49.3635,0.0024,3.0,1.007537,0.0565,0.14425
4,3250.0,15.0,1.0,11.0605,1.08075,49.3635,49.3617,0.0018,4.5,0.553689,0.0605,0.08075
...,...,...,...,...,...,...,...,...,...,...,...,...
103,4750.0,15.0,2.0,12.1850,2.08100,47.2419,47.2106,0.0313,15.0,10.826819,0.1850,0.08100
104,4750.0,20.0,2.0,12.1660,2.06300,47.2106,47.1822,0.0284,20.0,12.085559,0.1660,0.06300
105,5000.0,10.0,2.0,12.1390,2.07600,47.1822,47.1535,0.0287,10.0,6.269954,0.1390,0.07600
106,5000.0,15.0,2.0,12.1390,2.12500,47.1535,47.1256,0.0279,15.0,9.038995,0.1390,0.12500


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Spindle Speed             108 non-null    float64
 1   Feed (mm/min)             108 non-null    float64
 2   Tool Dia. (mm)            108 non-null    float64
 3   Length of cut actual(mm)  108 non-null    float64
 4   Actual Width(mm)          108 non-null    float64
 5   Wi                        108 non-null    float64
 6   Wf                        108 non-null    float64
 7   delta Weight              108 non-null    float64
 8   MRR ideal                 108 non-null    float64
 9   MRR actual                108 non-null    float64
 10  Delta L                   108 non-null    float64
 11  Delta W                   108 non-null    float64
dtypes: float64(12)
memory usage: 10.3 KB


## Data Preprocessing

In [6]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [7]:
X_train_full = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train_full =pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [8]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [9]:
X_train = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train=pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [10]:
X_val = val_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_val=pd.concat([val_set["Spindle Speed"], val_set["Feed (mm/min)"]], axis=1)

In [11]:
X_test = test_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_test=pd.concat([test_set["Spindle Speed"], test_set["Feed (mm/min)"]], axis=1)

In [12]:
X_train.shape

(68, 10)

In [13]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [14]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)

In [15]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [16]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 10)                21        
 on)                                                             
                                                                 
 batch_normalization (Batch  (None, 10)                40        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 500)               5500      
                                                                 
 batch_normalization_1 (Bat  (None, 500)               2000      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 300)               150300    
                                                        

In [18]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb],validation_data=(X_val,y_val))

Epoch 1/500




3/3 [==============================] - 3s 180ms/step - loss: 8146176.0000 - root_mean_squared_error: 2854.1504 - val_loss: 7872689.0000 - val_root_mean_squared_error: 2805.8311
Epoch 2/500
3/3 [==============================] - 0s 39ms/step - loss: 8133386.5000 - root_mean_squared_error: 2851.9094 - val_loss: 7865946.5000 - val_root_mean_squared_error: 2804.6294
Epoch 3/500
3/3 [==============================] - 0s 31ms/step - loss: 8124232.5000 - root_mean_squared_error: 2850.3040 - val_loss: 7857919.0000 - val_root_mean_squared_error: 2803.1982
Epoch 4/500
3/3 [==============================] - 0s 39ms/step - loss: 8116663.5000 - root_mean_squared_error: 2848.9758 - val_loss: 7849314.5000 - val_root_mean_squared_error: 2801.6628
Epoch 5/500
3/3 [==============================] - 0s 39ms/step - loss: 8109092.5000 - root_mean_squared_error: 2847.6467 - val_loss: 7839639.0000 - val_root_mean_squared_error: 2799.9355
Epoch 6/500
3/3 [==============================] - 0s 31ms/step - los

In [19]:
mse_score, rmse_score = model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 46ms/step - loss: 130042.4062 - root_mean_squared_error: 360.6139


In [20]:
X_new = X_train[:3]

In [21]:
X_train[:3]

,Tool Dia. (mm),Length of cut actual(mm),Actual Width(mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
63,3.0,13.328,3.112,48.3988,48.3622,0.0366,9.0,7.071112,0.328,0.112
31,2.0,12.089,2.234,49.1654,49.1434,0.0220,9.0,6.014213,0.089,0.234
71,3.0,13.156,3.179,48.1323,48.1152,0.0171,18.0,6.727256,0.156,0.179


In [22]:
model.predict(X_new)

1/1 [==============================] - 0s 222ms/step


array([[3839.229   ,   13.664449],
       [3385.2039  ,   15.849729],
       [4360.4624  ,   15.470887]], dtype=float32)

In [23]:
y_train[:3]

,Spindle Speed,Feed (mm/min)
63,3750.0,10.0
31,3250.0,15.0
71,4250.0,20.0


### Model Tuning

In [24]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [25]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

Trial 10 Complete [00h 01m 24s]
val_root_mean_squared_error: 173.96482849121094

Best val_root_mean_squared_error So Far: 122.36402130126953
Total elapsed time: 00h 08m 27s


In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [27]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [28]:
best_trial.summary()

Trial 03 summary
Hyperparameters:
n_hidden: 2
n_neurons: 492
learning_rate: 0.001715074355925934
Score: 122.36402130126953


In [29]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [30]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [31]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [32]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb, early_stopping_cb])

Epoch 1/200
3/3 [==============================] - 2s 70ms/step - loss: 134136.4844 - root_mean_squared_error: 366.2465
Epoch 2/200
3/3 [==============================] - 0s 16ms/step - loss: 84823.6250 - root_mean_squared_error: 291.2450
Epoch 3/200
3/3 [==============================] - 0s 16ms/step - loss: 142107.4844 - root_mean_squared_error: 376.9715
Epoch 4/200
3/3 [==============================] - 0s 31ms/step - loss: 122411.5312 - root_mean_squared_error: 349.8736
Epoch 5/200
3/3 [==============================] - 0s 31ms/step - loss: 138593.6094 - root_mean_squared_error: 372.2816
Epoch 6/200
3/3 [==============================] - 0s 16ms/step - loss: 118694.2656 - root_mean_squared_error: 344.5204
Epoch 7/200
3/3 [==============================] - 0s 23ms/step - loss: 146845.8438 - root_mean_squared_error: 383.2047
Epoch 8/200
3/3 [==============================] - 0s 16ms/step - loss: 145071.0156 - root_mean_squared_error: 380.8819
Epoch 9/200
3/3 [========================

In [33]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

Reusing TensorBoard on port 6006 (pid 9016), started 1 day, 5:58:23 ago. (Use '!kill 9016' to kill it.)

In [34]:
X_new

,Tool Dia. (mm),Length of cut actual(mm),Actual Width(mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
63,3.0,13.328,3.112,48.3988,48.3622,0.0366,9.0,7.071112,0.328,0.112
31,2.0,12.089,2.234,49.1654,49.1434,0.0220,9.0,6.014213,0.089,0.234
71,3.0,13.156,3.179,48.1323,48.1152,0.0171,18.0,6.727256,0.156,0.179


In [35]:
Final_model.predict(X_new)

1/1 [==============================] - 0s 251ms/step


array([[3640.1719   ,   10.8417225],
       [3442.5625   ,   15.817289 ],
       [4306.0273   ,   16.709497 ]], dtype=float32)

In [36]:
y_train[:3]

,Spindle Speed,Feed (mm/min)
63,3750.0,10.0
31,3250.0,15.0
71,4250.0,20.0


In [37]:
Final_model.save("Final_neural_network.h5")

e:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
from utils.Visualizations import plot_model
plot_model(Final_model)

In [15]:
model = tf.keras.models.load_model('Final_neural_network.h5')

In [16]:
model.evaluate(X_train, y_train)


3/3 [==============================] - 2s 32ms/step - loss: 15926.2520 - root_mean_squared_error: 126.1993


[15926.251953125, 126.19925689697266]

In [14]:
model.evaluate(X_test,y_test)


1/1 [==============================] - 4s 4s/step - loss: 35274.8281 - root_mean_squared_error: 187.8159


[35274.828125, 187.81594848632812]

In [13]:
model.predict(X_new)

NameError: name 'X_new' is not defined

In [17]:
from tensorflow.keras.utils import plot_model

# Plot the model architecture
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
